# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../Weather/weather_df.csv")
weather_df

,Unnamed: 0,Humidity,Cloudiness,Wind Speed,Country,City,Latitude,Longitude,Max Temperature,Date
0,0,73,30,7.70,NC,Poum,-20.2333,164.0167,75.54,1635136626
1,1,94,0,6.91,ZA,Port Elizabeth,-33.9180,25.5701,53.89,1635136627
2,2,92,78,3.98,CL,Lebu,-37.6167,-73.6500,53.10,1635136627
3,3,39,20,11.18,RU,Petropavlovsk-Kamchatskiy,53.0452,158.6483,41.59,1635136627
4,4,71,4,2.08,KI,Butaritari,3.0707,172.7902,82.35,1635136627
...,...,...,...,...,...,...,...,...,...,...
558,558,59,100,8.72,KZ,Emba,48.8264,58.1442,49.21,1635136782
559,559,95,62,9.10,RU,Anadyr,64.7500,177.4833,-9.80,1635136782
560,560,86,90,6.91,US,Monroe,43.1501,-77.6831,51.49,1635136672
561,561,44,9,1.01,IN,Phek,25.6667,94.5000,72.16,1635136782


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

gmaps.configure(api_key=g_key)

locations = weather_df[["Latitude","Longitude"]].astype(float)

In [4]:

figure_layout = {
    'width': '500px',
    'height': '460px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
image = gmaps.figure(layout=figure_layout)

In [8]:
humidity = weather_df["Humidity"] 
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)
image.add_layer(heat_layer)
plt.savefig("HeatMap.png")
image


Figure(layout=FigureLayout(border='1px solid black', height='460px', margin='0 auto 0 auto', padding='1px', wi…

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:

ideal_w= weather_df.loc[(weather_df["Max Temperature"]>= 70) & (weather_df["Max Temperature"]<= 85) 
                                 & (weather_df["Wind Speed"]<= 10) & (weather_df["Cloudiness"]>= 10)].dropna()
ideal_w


,Unnamed: 0,Humidity,Cloudiness,Wind Speed,Country,City,Latitude,Longitude,Max Temperature,Date
0,0,73,30,7.70,NC,Poum,-20.2333,164.0167,75.54,1635136626
5,5,70,87,0.31,SB,Buala,-8.1450,159.5921,84.15,1635136628
8,8,51,67,2.21,SS,Malakal,9.5334,31.6605,78.39,1635136629
11,11,78,20,8.05,TO,Vaini,-21.2000,-175.2000,78.96,1635136629
12,12,73,99,7.90,PH,Pandan,14.0453,124.1694,84.51,1635136630
...,...,...,...,...,...,...,...,...,...,...
548,548,98,100,4.00,BR,Presidente Médici,-11.1753,-61.9014,71.46,1635136779
549,549,71,100,5.84,PH,Aloleng,16.1307,119.7824,84.58,1635136780
553,553,75,11,3.53,BR,Santa Quitéria,-4.3319,-40.1567,74.62,1635136781
554,554,49,71,3.51,SD,Doka,13.5167,35.7667,79.65,1635136781


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:

hotel_df = pd.DataFrame(ideal_w)
hotel_df
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,Humidity,Cloudiness,Wind Speed,Country,City,Latitude,Longitude,Max Temperature,Date,Hotel Name
0,0,73,30,7.70,NC,Poum,-20.2333,164.0167,75.54,1635136626,
5,5,70,87,0.31,SB,Buala,-8.1450,159.5921,84.15,1635136628,
8,8,51,67,2.21,SS,Malakal,9.5334,31.6605,78.39,1635136629,
11,11,78,20,8.05,TO,Vaini,-21.2000,-175.2000,78.96,1635136629,
12,12,73,99,7.90,PH,Pandan,14.0453,124.1694,84.51,1635136630,
...,...,...,...,...,...,...,...,...,...,...,...
548,548,98,100,4.00,BR,Presidente Médici,-11.1753,-61.9014,71.46,1635136779,
549,549,71,100,5.84,PH,Aloleng,16.1307,119.7824,84.58,1635136780,
553,553,75,11,3.53,BR,Santa Quitéria,-4.3319,-40.1567,74.62,1635136781,
554,554,49,71,3.51,SD,Doka,13.5167,35.7667,79.65,1635136781,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
image.add_layer(marker_layer)
image